# importing packages

In [5]:
# pip install --upgrade torch torchvision
# use this for training

In [6]:
!pip install -qU sentence-transformers
!pip install -qU wikipedia-api
!pip install -qU clean-text[gpl]

!mkdir resources
!wget -q "https://github.com/sobhe/hazm/releases/download/v0.5/resources-0.5.zip" -P resources
!unzip -qq resources/resources-0.5.zip -d resources

!rm -rf /content/4ccae468eb73bf6c4f4de3075ddb5336
!rm -rf /content/preproc
!rm preprocessing.py utils.py
!mkdir -p /content/preproc
!git clone https://gist.github.com/4ccae468eb73bf6c4f4de3075ddb5336.git /content/preproc/
!mv /content/preproc/* /content/
!rm -rf /content/preproc

!pip install faiss-cpu
!pip install hazm

mkdir: cannot create directory ‘resources’: File exists
replace resources/chunker.model? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
Cloning into '/content/preproc'...
remote: Enumerating objects: 7, done.
remote: Total 7 (delta 0), reused 0 (delta 0), pack-reused 7
Unpacking objects: 100% (7/7), done.


In [7]:
import numpy as np 
import pandas as pd
import re
import os
import faiss
import hazm
from hazm import stopwords_list
import pickle
import requests
from termcolor import colored

from torch.utils.data import DataLoader
from sentence_transformers import InputExample, losses
import torch
import tensorflow as tf
from sentence_transformers import models, SentenceTransformer, util


SentenceTransformer_TrainedOnFarsTail_Path = 'm3hrdadfi/bert-fa-base-uncased-farstail-mean-tokens'
SentenceTransformer_TrainedOn_wikinli_Path = 'm3hrdadfi/bert-fa-base-uncased-wikinli-mean-tokens'
SentenceTransformer_TrainedOn_wikitriplet_Path = 'm3hrdadfi/bert-fa-base-uncased-wikitriplet-mean-tokens'

# Helpers

In [4]:

# phrases that need to be removed from titles
corona_phrases = ['کرونایی', 'کروناست' ,'کرونا', 'شیوع', 'بحران', 'ویروس',
                  'ویروس جدید', 'coronavirus', 'corona', 'کووید-19 ', 
                  'کووید', 'بیماری', 'بیمارانی', 'بیماران', '-۱۹', ' وی ', '19', '۱۹',
                  ' بیمار ', 'كرونا', 'كوويد', 'ويروس', r'(\s+)',]


normalizer = hazm.Normalizer()

def clean(text):
    """Cleans the titles for the semantic models"""
    for pattern in corona_phrases:
        text = re.sub(pattern, " ", text)

    text = re.sub(' +[\w] +', " ", text)
    text = normalizer.normalize(text)

    return text


#---------------------------------- get the results for this model-----------------------#

def get_resutls(questions, top_n):
    
    results = []
    for question in questions:

        print(question)
        # print('question type', type(question))


        # we give the cleaned question to the semantic model 
        question_cleaned = clean(question)

        question_emb = sentence_bert_embedder.encode(question_cleaned,
                                    convert_to_tensor=False,
                                    show_progress_bar = False)
        
        emb_que = np.array([question_emb])
        faiss.normalize_L2(emb_que)

        top_k = index.search(emb_que, top_n)
        indices = []
        scores = []

        # saving all the reults in a dictionary
        for score, idx in zip(top_k[0][0], top_k[1][0]):
            
            indices.append(idx)
            scores.append(score)

        results.append({'question':question,
                        'index':indices,
                        'score':scores})
    return results

# The Dataset

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:

data_address = '/content/drive/MyDrive/COVID-PSS.xls'
keys_address = '/content/drive/MyDrive/keywords_final_distilled_NE (1).pickle'
cleaned_titles_address = '/content/drive/MyDrive/title_cleaned_without_corona_2.pkl'
annotations_address = '/content/drive/MyDrive/all_annotations.csv'

df = pd.read_csv(data_address)
list_t = pd.read_pickle(cleaned_titles_address)

keywords = pd.read_pickle(keys_address)
keywords = [v for k,v in keywords.items()]



assert len(keywords) == len(df)
df['keywords'] = keywords
df.drop(columns=['img', 'link'], inplace=True)



In [10]:
corpora = []
for i in range(len(list_t)):

    keys = '[SEP]'.join(keywords[i])
    corpora.append(' '.join([list_t[i], keys]))

# The model


In [11]:
# preparing the model

def load_sentence_transformer(path):
    bert_model_s = models.Transformer(path, max_seq_length=512)
    pooling_model = models. Pooling(
        bert_model_s.get_word_embedding_dimension(),
        pooling_mode_mean_tokens = True,
        pooling_mode_cls_token = False,
        pooling_mode_max_tokens = False,
    )
    model = SentenceTransformer(modules= [bert_model_s, pooling_model])
    return model

sentence_bert_embedder = load_sentence_transformer(SentenceTransformer_TrainedOnFarsTail_Path)

# Creating Embeddings

In [12]:
%%time
# creating corpus embeddings

corpus_embeddings = sentence_bert_embedder.encode(corpora,
                                                  convert_to_tensor=True,
                                                  show_progress_bar=True,
                                                  batch_size = 256)


CPU times: user 21.7 s, sys: 18.1 s, total: 39.8 s
Wall time: 49.1 s


# Sample Qustions

In [24]:
# creating sample question embeddings

question = str('باردار باردار باردار باردار باردار باردار ')
question_emb = sentence_bert_embedder.encode(question,
                               convert_to_tensor=False,
                               show_progress_bar = False)

Searching with FAISS

In [25]:
## Similarity Scores with FAISS

index = faiss.index_factory(768, "Flat", faiss.METRIC_INNER_PRODUCT)
# corpus
emb_cor = np.array(corpus_embeddings)
faiss.normalize_L2(emb_cor)

# query
index.add(emb_cor)
emb_que = np.array([question_emb])
faiss.normalize_L2(emb_que)

top_k = index.search(emb_que, 100)

In [26]:
sentence_bert_WikiNli_indices = []
sentence_bert_WikiNli_scores = []

print(question)
print('Top results!')
i=0
for score, idx in zip(top_k[0][0], top_k[1][0]):
    print(colored(f'{i}th', 'blue'),' corpus with score', colored( f'{score:.2f}:\n', 'blue'),  corpora[idx])
    i+=1
    sentence_bert_WikiNli_indices.append(idx)
    sentence_bert_WikiNli_scores.append(score)

باردار 
Top results!
0th  corpus with score 0.75:
 تاج هم را شکست داد بیمارستان مرخص[SEP]بهبود نسبی[SEP]حال خبرها
1th  corpus with score 0.74:
 مدیرعامل بانک مسکن بر اثر در گذشت اثر ابتلا[SEP]گذشت ایلنا ساعاتی[SEP]در گذشت ایلنا ساعاتی
2th  corpus with score 0.71:
 در مقابل تاب می‌آوریم؟ واکنشهای احساسی بیشتری[SEP]حال همکاران بیمارم[SEP]تاب اوری شان[SEP]توان و تاباوری بیشتری[SEP]سپری شدن شرایط سخت و مصیبت زندگی خود[SEP]تاباوری یا تاباوری روانی[SEP]خود حرف زدن[SEP]کمک کننده[SEP]تاب اوری[SEP]انجام کارهای عقب[SEP]نام تاب اوری[SEP]خانواده پرجمعیتی[SEP]چند ماهی[SEP]مشکلات و سختیها[SEP]دیگر نمیکشم‌ها[SEP]خانه دایم[SEP]کارمند یک شرکت خصوصی[SEP]همان تاباوری خودشان[SEP]مدت شنیدهام[SEP]مادرت قرنطینه[SEP]همه جای دنیا[SEP]شرایط باب گفتوگو[SEP]پایین امده غمگین[SEP]دست اوردن تفکر و مهارتهای خود[SEP]گیر افتادهایم دکتر روانشناس[SEP]برخی تاباوری شان
3th  corpus with score 0.70:
 چند دقیقه در کنار فرد مبتلا بمانیم می‌گیریم؟ سه هزار قطره[SEP]طور قابل توجهی[SEP]فردی مبتلا[SEP]دفتر کار[SEP]اغلب قطرهها[SEP]ر

In [ ]:
question

# Sampling in Batches

In [13]:
questions = pd.read_pickle('/content/drive/MyDrive/CoPer paper-Models/Sample Queries/Titles_with_Corona.pkl')

Don't put faiss itself in the function it'll give wrong results and indices

In [14]:
# create the basic faiss for corpus embeddings
index = faiss.index_factory(768, "Flat", faiss.METRIC_INNER_PRODUCT)
emb_cor = np.array([i.numpy() for i in corpus_embeddings])
faiss.normalize_L2(emb_cor)
index.add(emb_cor)



In [15]:
results = get_resutls(questions, top_n = 50)

آیا زنان باردار بیشتر در معرض خطر ابتلا به بیماری کروناهستند 
من باردار هستم چگونه می‌توانم از خودم در مقابل بیماری کرونامحافظت کنم
آیا زنان باردار باید برای کروناآزمایش شوند
آیا کرونامی تواند از مادر باردار به نوزاد متولد نشده جنین یا نوزاد تازه متولد شده منتقل شود
در دوران بارداری حین زایمان چه مراقبت‌هایی باید انجام شود
آیا زنان باردار مبتلا به کرونایا مشکوک به این بیماری نیاز به سزارین دارند
آیا مادر مبتلا به کرونامی تواند به نوزاد خود شیر بدهد
آیا در صورت مبتلا شدن به کووید۱۹ می‌توانم کودک خود را در آغوش بگیرم
من به کرونامبتلا هستم حال عمومی مساعدی ندارم که بتوانم به کودک خود مستقیما شیر بدهم چه باید بکنم
آیا خشک­ کن­های دست در نابودی ویروس جدید کرونا موثر هستند
آیا گندزداییضدعفونی با لامپ ماورای بنفش ویروس جدید کرونا را از بین می­برد
آیا سنجش حرارت بدن اندازه گیری تب در شنسایی افراد مبتلا به عفونت ویروس جدید کرونا موثر است
آیا اسپری کردن الکل یا محلول‌های حاوی کلر روی تمام بدن می­تواند ویروس جدید کرونا را نابود کند
آیا واکسنهای ذات­الریه در برابر ویروس جدید کرونا موثر هستند
آیا ش

# Saving all in a pickle

In [16]:
with open('/content/drive/MyDrive/CoPer paper-Models/Results/sbert-farsTail.pkl', 'wb') as f:
    pickle.dump(results, f)